In [1]:
import fastai; fastai.__version__

'1.0.12.dev0'

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
fns_trn=['1.jpg', '2.jpg']
lbls_trn=[['a', 'b'], ['c']]

fns_val=['3.jpg']
lbls_val=[['a', 'd']]

In [4]:
train_ds=ImageMultiDataset(fns_trn, lbls_trn)

In [5]:
valid_ds=ImageMultiDataset(fns_val, lbls_val, train_ds.classes)

KeyError: 'd'

Previously proposed fix in this closed [PR](https://github.com/fastai/fastai/pull/956). We would need a proper way to handle this. 

In [6]:
class NewImageMultiDataset(LabelDataset):
    
    def __init__(self, fns:FilePathList, labels:ImgLabels, classes:Optional[Classes]=None):
        self.classes = ifnone(classes, uniqueify(np.concatenate(labels)))
        self.class2idx = {v:k for k,v in enumerate(self.classes)}
        self.x = np.array(fns)
        self.y = [np.array([self.class2idx[o] for o in l], dtype=np.int64)
                  for l in labels if l in self.classes]
        self.loss_func = F.binary_cross_entropy_with_logits

In [7]:
train_ds=NewImageMultiDataset(fns_trn, lbls_trn)
valid_ds=NewImageMultiDataset(fns_val, lbls_val, train_ds.classes)

In [8]:
train_ds, valid_ds

(NewImageMultiDataset of len 2, NewImageMultiDataset of len 1)

More discussion on the forum

https://forums.fast.ai/t/how-should-imagemultidataset-handle-labels-in-valid-ds-unseen-by-train-ds/27869/2?u=yang-zhang

I think it is best to make no changes to the code. And when creating `train_ds`, if you think this “val-labels-not-in-train” situation might happen, pass the union of the labels of training and validation labels.

In [16]:
lbs_all = set([j for i in lbls_trn+lbls_val for j in i])

In [17]:
train_ds=ImageMultiDataset(fns_trn, lbls_trn, lbs_all)

In [18]:
valid_ds=ImageMultiDataset(fns_val, lbls_val, lbs_all)

In [19]:
train_ds.classes, valid_ds.classes

({'a', 'b', 'c', 'd'}, {'a', 'b', 'c', 'd'})